In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import savemat
import shutil
import dartsort
import dartsort.vis as dartvis

In [3]:
darts_dir = Path("~/proj/measims/dart_recordings").expanduser()

In [5]:
dart_out_dir = Path("~/proj/measims/dart_out").expanduser()
dart_out_dir.mkdir(exist_ok=True)

In [4]:
list(darts_dir.iterdir())

[PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_non-rigid_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_non-rigid_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_uniform_modulat

In [23]:
cfg = dartsort.DARTsortConfig(
    matching_iterations=0,
    subtract_only=True,
    # hacking the times to use our NN at 32kHz
    waveform_config=dartsort.WaveformConfig(ms_before=1.32, ms_after=2.48),
)

In [22]:
import pickle
def loadpkl(p):
    with open(p, "rb") as f:
        return pickle.load(f)

In [ ]:
for p in darts_dir.iterdir():
    rec = si.read_binary_folder(p)
    out_dir = dart_out_dir / p.stem
    st = dartsort.dartsort(
        rec, out_dir, cfg=cfg, n_jobs_gpu=2
    )
    me = loadpkl(out_dir / "motion_est.pkl")
    dartvis.visualize_sorting(
        rec,
        st,
        out_dir / "vis",
        motion_est=me,
        make_sorting_summaries=False,
        make_animations=False,
        make_unit_summaries=False,
    )

Fit subtraction denoisers 0.9s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Fit features 0.9s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Subtraction 0.9s/it [spk/it=%%%]:   0%|          | 0/640 [00:00<?, ?it/s]

...

Subtraction 0.9s/it [spk/it=%%%]:   0%|          | 0/640 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Localization:   0%|          | 0/23 [00:00<?, ?it/s]

Cross correlation:   0%|          | 0/3 [00:00<?, ?it/s]

Solve:   0%|          | 0/2 [00:00<?, ?it/s]